In [363]:
import bs4, requests, re, bibtexparser

In [644]:
mode_layout = 'flat'
article_id = '1882373'
expformat = 'bibtex'

domain = 'https://dl.acm.org/'
article = 'citation.cfm?id={0}&'.format(article_id)
parametrs = 'preflayout={0}'.format(mode_layout)
url = domain + article + parametrs
# https://dl.acm.org/downformats.cfm?id=2492591&parent_id=&expformat=bibtex
url_download_bibtex = domain + 'downformats.cfm?id={0}&parent_id=&expformat={1}'.format(article_id, expformat)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
file = open('test.html', 'wb')

print(url, url_download_bibtex, sep='\n')

https://dl.acm.org/citation.cfm?id=1882373&preflayout=flat
https://dl.acm.org/downformats.cfm?id=1882373&parent_id=&expformat=bibtex


In [645]:
response = requests.get(url=url, headers=headers)
soup = bs4.BeautifulSoup(response.text, 'lxml')
bibtex_file = open('bibtex.bib', 'w').write(requests.get(url=url_download_bibtex, headers=headers).text)
bibtex_file = open('bibtex.bib', 'r')
bibtex_dict = bibtexparser.load(bibtex_file).entries

In [646]:
article_data = {}

article_data['id'] = article_id
article_data['url'] = url
article_data['title'] = bibtex_dict[0].get('title', None)
article_data['doi'] = bibtex_dict[0].get('doi', None) 
article_data['year'] = bibtex_dict[0].get('year', None)

article_data, bibtex_dict

({'doi': '10.1145/1882362.1882373',
  'id': '1882373',
  'title': 'Managing Technical Debt in Software-reliant Systems',
  'url': 'https://dl.acm.org/citation.cfm?id=1882373&preflayout=flat',
  'year': '2010'},
 [{'ENTRYTYPE': 'inproceedings',
   'ID': 'Brown:2010:MTD:1882362.1882373',
   'acmid': '1882373',
   'address': 'New York, NY, USA',
   'author': 'Brown, Nanette and Cai, Yuanfang and Guo, Yuepu and Kazman, Rick and Kim, Miryung and Kruchten, Philippe and Lim, Erin and MacCormack, Alan and Nord, Robert and Ozkaya, Ipek and Sangwan, Raghvinder and Seaman, Carolyn and Sullivan, Kevin and Zazworka, Nico',
   'booktitle': 'Proceedings of the FSE/SDP Workshop on Future of Software Engineering Research',
   'doi': '10.1145/1882362.1882373',
   'isbn': '978-1-4503-0427-6',
   'keywords': 'cost-benefit analysis, design decision trade-off, large-scale system development, software metrics, technical debt',
   'location': 'Santa Fe, New Mexico, USA',
   'numpages': '6',
   'pages': '47--5

In [647]:
divmain = soup.find('div', id='divmain')

authors_tags = divmain.find_all('td', {'style': 'padding-right:3px;', 'valign': 'top',  'nowrap': 'nowrap'})
authors = []

for author_tag in authors_tags:
    name = author_tag.text
    authors.append({'name': name.strip(),
                    'url': domain + author_tag.find('a')['href']})
    
article_data['authors'] = authors
article_data

{'authors': [{'name': 'Nanette Brown',
   'url': 'https://dl.acm.org/author_page.cfm?id=81472641102&coll=DL&dl=ACM&trk=0'},
  {'name': 'Yuanfang Cai',
   'url': 'https://dl.acm.org/author_page.cfm?id=81350599209&coll=DL&dl=ACM&trk=0'},
  {'name': 'Yuepu Guo',
   'url': 'https://dl.acm.org/author_page.cfm?id=81375596470&coll=DL&dl=ACM&trk=0'},
  {'name': 'Rick Kazman',
   'url': 'https://dl.acm.org/author_page.cfm?id=81100391310&coll=DL&dl=ACM&trk=0'},
  {'name': 'Miryung Kim',
   'url': 'https://dl.acm.org/author_page.cfm?id=81335492829&coll=DL&dl=ACM&trk=0'},
  {'name': 'Philippe Kruchten',
   'url': 'https://dl.acm.org/author_page.cfm?id=81414594403&coll=DL&dl=ACM&trk=0'},
  {'name': 'Erin Lim',
   'url': 'https://dl.acm.org/author_page.cfm?id=81472652512&coll=DL&dl=ACM&trk=0'},
  {'name': 'Alan MacCormack',
   'url': 'https://dl.acm.org/author_page.cfm?id=81100187120&coll=DL&dl=ACM&trk=0'},
  {'name': 'Robert Nord',
   'url': 'https://dl.acm.org/author_page.cfm?id=81332518853&coll=D

In [648]:
layout = soup.find('div', {'class': 'layout'})
flatbody = layout.find('div', {'class': 'flatbody'})

abstract = flatbody.text.strip()
article_data['abstract'] = abstract

article_data

{'abstract': 'Delivering increasingly complex software-reliant systems demands better ways to manage the long-term effects of short-term expedients. The technical debt metaphor is gaining significant traction in the agile development community as a way to understand and communicate such issues. The idea is that developers sometimes accept compromises in a system in one dimension (e.g., modularity) to meet an urgent demand in some other dimension (e.g., a deadline), and that such compromises incur a "debt": on which "interest" has to be paid and which the "principal" should be repaid at some point for the long-term health of the project. We argue that the software engineering research community has an opportunity to study and improve this concept. We can offer software engineers a foundation for managing such trade-offs based on models of their economic impacts. Therefore, we propose managing technical debt as a part of the future research agenda for the software engineering field.',
 '

In [649]:
td = soup.find('td', string='Conference').nextSibling.nextSibling
venue = td.strong.text.strip()

url_conference = td.a['href']

article_data['venue'] = {'name': venue,
                        'url': domain + url_conference}

article_data

{'abstract': 'Delivering increasingly complex software-reliant systems demands better ways to manage the long-term effects of short-term expedients. The technical debt metaphor is gaining significant traction in the agile development community as a way to understand and communicate such issues. The idea is that developers sometimes accept compromises in a system in one dimension (e.g., modularity) to meet an urgent demand in some other dimension (e.g., a deadline), and that such compromises incur a "debt": on which "interest" has to be paid and which the "principal" should be repaid at some point for the long-term health of the project. We argue that the software engineering research community has an opportunity to study and improve this concept. We can offer software engineers a foundation for managing such trade-offs based on models of their economic impacts. Therefore, we propose managing technical debt as a part of the future research agenda for the software engineering field.',
 '

In [650]:
flatbody = soup.find_all('div', {'class': 'flatbody'})
cited_by = []
for a in flatbody[3].find_all('a'):
    cited_by.append(a['href'].split('=')[-1])
    
article_data['cited_by'] = cited_by

article_data

{'abstract': 'Delivering increasingly complex software-reliant systems demands better ways to manage the long-term effects of short-term expedients. The technical debt metaphor is gaining significant traction in the agile development community as a way to understand and communicate such issues. The idea is that developers sometimes accept compromises in a system in one dimension (e.g., modularity) to meet an urgent demand in some other dimension (e.g., a deadline), and that such compromises incur a "debt": on which "interest" has to be paid and which the "principal" should be repaid at some point for the long-term health of the project. We argue that the software engineering research community has an opportunity to study and improve this concept. We can offer software engineers a foundation for managing such trade-offs based on models of their economic impacts. Therefore, we propose managing technical debt as a part of the future research agenda for the software engineering field.',
 '